# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-09-01	Numerical simulation of surfactant-laden emulsion formation in an un-baffled stirred vessel	Chemical Engineering Journal		 Liang, Fuyue, et al. "Numerical simulation of surfactant-laden emulsion formation in an un-baffled stirred vessel." Chemical Engineering Journal 472(2023).	https://doi.org/10.1016/j.cej.2023.144807	
2023-11-01	On the dispersion dynamics of liquid–liquid surfactant-laden flows in a SMX static mixer	Chemical Engineering Journal		Valdes, J. P., Kahouadji, L., Liang, F., Shin, S., Chergui, J., Juric, D., & Matar, O. K. (2023). "On the dispersion dynamics of liquid–liquid surfactant-laden flows in a SMX static mixer." Chemical Engineering Journal, 475, 146058.	https://doi.org/10.1016/j.cej.2023.146058	
2023-04-15	Direct numerical simulations of liquid–liquid dispersions in a SMX mixer under different inlet conditions	Chemical Engineering Journal		Valdes, J. P., Kahouadji, L., **Liang, F.**, Shin, S., Ch

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-09-01,Numerical simulation of surfactant-laden emuls...,Chemical Engineering Journal,NaN,"Liang, Fuyue, et al. ""Numerical simulation of...",https://doi.org/10.1016/j.cej.2023.144807,NaN
1,2023-11-01,On the dispersion dynamics of liquid–liquid su...,Chemical Engineering Journal,NaN,"Valdes, J. P., Kahouadji, L., Liang, F., Shin,...",https://doi.org/10.1016/j.cej.2023.146058,NaN
2,2023-04-15,Direct numerical simulations of liquid–liquid ...,Chemical Engineering Journal,NaN,"Valdes, J. P., Kahouadji, L., **Liang, F.**, S...",https://doi.org/10.1016/j.cej.2023.142248,NaN
3,2022-09-29,Numerical study of oil–water emulsion formatio...,Flow,NaN,"Liang, Fuyue, et al. ""Numerical study of oil–w...",https://doi.org/10.1017/flo.2022.27,NaN
4,2022-09-22,The transition to aeration in turbulent two-ph...,Flow,NaN,"Kahouadji, Lyes, Fuyue Liang, Juan P. Valdes, ...",https://doi.org/10.1017/flo.2022.24,NaN
5,2021-04-22,Enhancement of Damaged-image prediction throug...,Royal Society Open Science,NaN,"Carrillo, José A., Serafim Kalliadasis, Fuyue ...",https://doi.org/10.1098/rsos.201294,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2015-10-01-paper-title-number-3.md j.cej.2023.144807.md
flo.2022.24.md                     j.cej.2023.146058.md
flo.2022.27.md                     rsos.201294 .md
j.cej.2023.142248.md


In [4]:
!cat ../_publications/2015-10-01-paper-title-number-3.md
!cat ../_publications/flo.2022.24.md



---
title: "Paper Title Number 3"
collection: publications
permalink: /publication/2015-10-01-paper-title-number-3
excerpt: 'This paper is about the number 3. The number 4 is left for future work.'
date: 2015-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper3.pdf'
citation: 'Your Name, You. (2015). &quot;Paper Title Number 3.&quot; <i>Journal 1</i>. 1(3).'
---
This paper is about the number 3. The number 4 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper3.pdf)

Recommended citation: Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).---
title: "The transition to aeration in turbulent two-phase mixing in stirred vessels"
collection: publications
permalink: /publication/2022-09-22-https://doi.org/10.1017/flo.2022.24
date: 2022-09-22
venue: 'Flow'
citation: 'Kahouadji, Lyes, Fuyue Liang, Juan P. Valdes, Seungwon Shin, Jalel Chergui, Damir Juric, Richard V. Craster, and Omar K. Matar. &quot;The trans